In [1]:
### Packages
include("/home/moon/sfortune/spinevo/spinevo.jl")

Loading Functions:


'align_plane'   'orbit_j'   

'borders_tan'   'borders_arctan'   'reduce_n_particles'   'find_felixID'   'find_ISUB'   'ssFPfinder'   'cartesian_to_spherical'   'aligner'   'find_merging_progenitors'   'plot_group'   

'caker'   


In [2]:
### Settings

box         = "/HydroSims/Magneticum/Box4/uhr_test"
input_dir   = "/home/moon/sfortune/spinevo/halostories_v20211007_min0.0Gyr"
output_dir  = "/home/moon/sfortune/spinevo/mergerimpact"
mass_ST_thr = 1e10

storyfilelist   = readdir(input_dir)
print("Done!")


Done!

In [3]:
assembly_DM     = load(joinpath(output_dir, "assembly_Mstar_$(mass_ST_thr).jld"), "assembly_DM")
println("Done 1 / 3")
assembly_GAS    = load(joinpath(output_dir, "assembly_Mstar_$(mass_ST_thr).jld"), "assembly_GAS")
println("Done 2 / 3")
assembly_STARS  = load(joinpath(output_dir, "assembly_Mstar_$(mass_ST_thr).jld"), "assembly_STARS")
println("Done 3 / 3")


Done 1 / 3


Done 2 / 3


Done 3 / 3


In [4]:
#array of max merger indices
indices_main_mm = missings(Int64, 0)
indices_MMdm    = missings(Int64, 0)
for i in 1:length(assembly_DM["M2_felix"])
    if sum( assembly_DM["Merger_Map"][2, 1+sum(assembly_DM["N_MERGERS"][1:i-1]) : sum(assembly_DM["N_MERGERS"][1:i])] ) > 0. # is there a merger?
        global indices_MMdm    = vcat( indices_MMdm, sum(assembly_DM["N_MERGERS"][1:i-1]) + argmax( assembly_DM["Merger_Map"][2,1+sum(assembly_DM["N_MERGERS"][1:i-1]):sum(assembly_DM["N_MERGERS"][1:i])] ) )
        global indices_main_mm = vcat( indices_main_mm, i)
    end
end
m2_felix    = assembly_DM["Merger_Map"][4,indices_MMdm]
m2_mm       = assembly_DM["Merger_Map"][2,indices_MMdm]
println("$(size(m2_felix))   $(size(m2_mm))")
println(length(indices_MMdm))
println( length(indices_MMdm)-count(ismissing, indices_MMdm) )
println(count(ismissing, log10.(m2_felix ./ m2_mm) ) )
println(count(ismissing, replace( log10.(m2_felix ./ m2_mm), missing=>NaN ) ) )
println(count(isnan, replace( log10.(m2_felix ./ m2_mm), missing=>NaN ) ) )

(2278,)   (2278,)
2278


2278


0
0


0


In [5]:
#count(isnan,assembly_DM["Merger_Map"][4,indices_MMdm] ./ assembly_DM["Merger_Map"][2,indices_MMdm])
count(iszero,assembly_DM["Merger_Map"][2,indices_MMdm])

0

In [6]:
bin_factor  = 20
ϕmin        = 0#minimum(assembly_STARS["ϕ_flip"])
ϕmax        = 180#maximum(assembly_STARS["ϕ_flip"])
ymin        = 0.1 #-2 #minimum(assembly_DM["Merger_Map"][4,indices_MMdm] ./ assembly_DM["Merger_Map"][2,indices_MMdm])
ymax        = 20 #6 #maximum(assembly_DM["Merger_Map"][4,indices_MMdm] ./ assembly_DM["Merger_Map"][2,indices_MMdm])
matplotlib  = pyimport("matplotlib")
axes_grid1  = pyimport("mpl_toolkits.axes_grid1")
pltcolors   = pyimport("matplotlib.colors")
matplotlib.use("qt5agg")




fig, ax = subplots()

h = ax.hexbin( replace(assembly_STARS["ϕ_flip"][indices_main_mm], missing=>NaN), 
    replace(assembly_DM["Merger_Map"][4,indices_MMdm] ./ assembly_DM["Merger_Map"][2,indices_MMdm], 0.0=>NaN), #yscale="log", 
    gridsize=bin_factor,
    extent=[ϕmin,ϕmax, ymin,ymax], 
    cmap="BuPu", zorder=1, norm=pltcolors.LogNorm() )

#ax.set_yticks([1e-2,1e-1,1e0,1e1,1e2,1e3,1e4,1e5,1e6])
ax.set_xlabel("Flip Angle ϕ [°]")
ax.set_ylabel("DM Mass Ratio")
ax.set_title("Most Massive Merger")
#ax.grid()
fig.colorbar(h, ax=ax)

scale=0.7

fig.set_size_inches(16scale, 9*scale)
fig.tight_layout()
show()

#fig.savefig(joinpath(@__DIR__, "plots/M2mostmassive_vs_flipangles_MSTARSgt$(mass_ST_thr).png"), bbox_inches="tight", pad_inches=.1)


PyCall.PyError: PyCall.PyError("\$(Expr(:escape, :(ccall(#= /home/moon/sfortune/.julia/packages/PyCall/3fwVL/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))", PyObject(Ptr{PyCall.PyObject_struct} @0x00007f9a4667a3a0), PyObject(Ptr{PyCall.PyObject_struct} @0x00007f9a35f8bdd0), PyObject(Ptr{PyCall.PyObject_struct} @0x00007f9a35f98100))

In [7]:
# δj collection

δj_abs      = missings(Float64, 0)
δj_rel      = missings(Float64, 0)
δbval       = missings(Float64, 0)
indices_j   = missings(Int64, 0)
indices_j_main_mm   = missings(Int64, 0)
indices_j_MMdm      = missings(Int64, 0)
indices_giant_mergers   = missings(Int64, 0)

for i in 1:length(assembly_STARS["δj_main"][1,:])
    if count(ismissing, assembly_STARS["δj_main"][:,i]) == 0 # dj data available?
        δj_abs      = vcat( δj_abs, norm(assembly_STARS["δj_main"][:,i]) )
        δbval       = vcat( δbval, assembly_STARS["δBVAL"][i] )
        δj_rel      = vcat( δj_rel, norm(assembly_STARS["δj_main"][:,i]) / ( norm(assembly_STARS["j_main"][:,i] - assembly_STARS["δj_main"][:,i]) ) )
        indices_j   = vcat( indices_j, i )
        if i in indices_main_mm
            indices_j_main_mm   = vcat(indices_j_main_mm, i)
            indices_j_MMdm      = vcat(indices_j_MMdm, sum(assembly_DM["N_MERGERS"][1:i-1]) + argmax( assembly_DM["Merger_Map"][2,1+sum(assembly_DM["N_MERGERS"][1:i-1]):sum(assembly_DM["N_MERGERS"][1:i])] ) )
            if assembly_DM["Merger_Map"][4,indices_j_MMdm[end]] / assembly_DM["Merger_Map"][2,indices_j_MMdm[end]] < 1e-3
                println("Halo $(assembly_DM["ID_ISUB"][indices_j_main_mm[end]]) with    ---   Snap $(assembly_DM["Merger_Map"][5,indices_j_MMdm[end]])   ---   Ratio $(assembly_DM["Merger_Map"][4,indices_j_MMdm[end]] / assembly_DM["Merger_Map"][2,indices_j_MMdm[end]])")
                indices_giant_mergers   = vcat(indices_giant_mergers, i)
            end
        end
    else
        print(" ")
    end
end

println("\n\n$(count(ismissing, δj_rel))   $(count(ismissing, assembly_STARS["ϕ_flip"][indices_j]))")
println("$(size(δj_rel))   $(size(assembly_STARS["ϕ_flip"][indices_j]))")

 Halo 17615 with    ---   Snap 113.0   ---   Ratio 0.0009032082240927281
 Halo 9683 with    ---   Snap 128.0   ---   Ratio 3.5334894522557883e-6
  

 Halo 2323 with    ---   Snap 79.0   ---   Ratio 0.0
   

Halo 12334 with    ---   Snap 131.0   ---   Ratio 0.0
  

 Halo 15394 with    ---   Snap 116.0   ---   Ratio 0.0002877531122384094
 

 

0   0


(13527,)   (13527,)


In [75]:

#temp_mass2plot = convert( Array{Float64},assembly_STARS["δM2_felix"][indices_j] )
temp_mass2plot = assembly_STARS["M2_MM"][indices_j]
replace!(temp_mass2plot, missing => NaN)
for i in 1:length(temp_mass2plot)
    if temp_mass2plot[i] <= 0.
        temp_mass2plot[i] = NaN
    end
end
#convert(Array{Float64},temp_mass2plot)
#temp_mass2plot[temp_mass2plot. <= 0] .= NaN
#@show temp_mass2plot

In [76]:
# δj vs b-value

bin_factor  = 50
ϕmin        = 0
ϕmax        = 180
ymin        = -2
ymax        = 6
matplotlib  = pyimport("matplotlib")
axes_grid1  = pyimport("mpl_toolkits.axes_grid1")
pltcolors   = pyimport("matplotlib.colors")
matplotlib.use("qt5agg")


#temp_mass2plot = assembly_STARS["δM2_felix"][indices_j]
#temp_mass2plot[temp_mass2plot. <= 0] .= NaN
#println("$(size(temp_mass2plot))")


fig, ax = subplots()

#h = ax.hexbin( assembly_STARS["δM2_felix"][indices_j], 
h = ax.hexbin( temp_mass2plot, 
    δj_abs, yscale="log", xscale="log", 
    gridsize=bin_factor,
    #extent=[ϕmin,ϕmax, ymin,ymax], 
    cmap="BuPu", zorder=1, norm=pltcolors.LogNorm() )

#ax.set_yticks([1e-2,1e-1,1e0,1e1,1e2,1e3,1e4,1e5,1e6])
ax.set_xlabel("Most Massive M2_star [M_⊙]")
#ax.set_ylabel("|δj_star| / |j_star,t1|")
ax.set_ylabel("|δj_star|")
#ax.set_title("Most Massive Merger")
#ax.grid()
fig.colorbar(h, ax=ax)

scale=0.7

fig.set_size_inches(16*scale, 9*scale)
fig.tight_layout()
show()

#fig.savefig(joinpath(@__DIR__, "plots/djabs_vs_M2mm_MSTARSgt$(mass_ST_thr).png"), bbox_inches="tight", pad_inches=.1)
